In [1]:
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

sys.path.append("../src/")
from Network_Validator_Main import main

DTALite, version 0.8.1



In [2]:
%run ../src/Network_Validator_Main.py

Subdirectories:
../data/
GMNS Network Validator
Working directory: ../data/
Using node file: ../data/node.csv
Using link file: ../data/link.csv
Using demand file: ../data/demand.csv
Validation level: 7

Running validation at Readiness Level 7...

SUCCESS: Zone centroids are correctly listed in non-decreasing order at the beginning of the file

First through node identified as node_id=235

Found 234 zone centroids out of 8521 total nodes

First through node identified as node_id=235

Found 235 unique zone IDs in node file
Found 234 unique zone IDs in demand file

SUCCESS: All zones in demand file exist in node file
Copied settings.csv and NEXTA.exe to ../data
File found: ../data\od_performance.csv

OD pairs with impossibly low distance ratios (network distance shorter than straight line):
  O=175, D=208, Network=0.01km, Straight=1.84km, Ratio=0.002998
  O=208, D=175, Network=0.01km, Straight=1.84km, Ratio=0.004350
  O=78, D=7, Network=0.02km, Straight=2.04km, Ratio=0.007946
  O=7, D=78,